In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import import_ipynb
#import Segmentation
import time
import preds

importing Jupyter notebook from preds.ipynb
importing Jupyter notebook from Captcha_seg.ipynb
1080


/home/bg/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Train on 864 samples, validate on 216 samples
Epoch 1/20
864/864 [==============================] - 7s 8ms/step - loss: 3.5887 - acc: 0.0278 - val_loss: 3.6100 - val_acc: 0.0185
Epoch 2/20
864/864 [==============================] - 6s 7ms/step - loss: 3.4564 - acc: 0.0671 - val_loss: 3.2577 - val_acc: 0.1250
Epoch 3/20
864/864 [==============================] - 6s 7ms/step - loss: 2.8091 - acc: 0.1991 - val_loss: 2.6887 - val_acc: 0.2407
Epoch 4/20
864/864 [==============================] - 6s 7ms/step - loss: 2.0242 - acc: 0.3854 - val_loss: 2.1871 - val_acc: 0.3519
Epoch 5/20
864/864 [==============================] - 6s 7ms/step - loss: 1.4236 - acc: 0.5833 - val_loss: 1.9082 - val_acc: 0.4769
Epoch 6/20
864/864 [==============================] - 6s 7ms/step - loss: 0.9584 - acc: 0.6863 - val_loss: 1.7891 - val_acc: 0.4861
Epoch 7/20
864/864 [==============================] - 6s 7ms/step - loss: 0.6296 - acc: 0.8102 - val_loss: 1.5796 - val_acc: 0.6065
Epoch 8/20
864/864 [==========

In [2]:
CATEGORIES = set()
names =[]
for i in os.listdir('test/'):
    names.append(i.split('.')[0])
for i in names:
    for x in list(i):
        CATEGORIES.add(x)
CATEGORIES=list(CATEGORIES)
print(CATEGORIES)

['N', '2', '5', '6', 'D', 'P', 'G', 'L', '7', '1', 'X', 'E', 'T', 'S', 'J', 'Z', 'F', 'H', 'U', 'R', 'K', 'B', '4', 'V', 'A', 'Y', '9', 'C', 'M', '8', '3', 'Q', 'W']


In [3]:
#print(len(train_data))

In [4]:
import random

#random.shuffle(train_data)

In [5]:
#for sample in train_data[:10]:
#    print(sample[1])

In [6]:
X = []
y = []

In [7]:
#for features, label in train_data:
#    X.append(features)
#    y.append(label)
    
#X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

#import pickle

#pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()
pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

pickle_in= open("X.pickle", "rb")
X = pickle.load(pickle_in)
X.shape[1:]

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout,Activation, Flatten, Conv2D, MaxPooling2D

X = pickle.load(open("X.pickle","rb"))
y = pickle.load(open("y.pickle","rb"))

X = X/255.0

model = Sequential()

model.add(Conv2D(64,(3,3),input_shape= X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))


model.add(Dense(19))
model.add(Activation("softmax"))

from keras.utils import to_categorical
y_binary = to_categorical(y)

model.compile(loss="categorical_crossentropy",
             optimizer="adam",
             metrics=['accuracy'])

model.fit(X,y_binary,batch_size=128, epochs=20, validation_split = 0.2)


#loss_value, metrics_value = model.evaluate(X,y_binary,batch_size=10)

#print(loss_value, metrics_value)

In [8]:
import random

In [9]:
def similarity(a,b):
    i =0
    for x,y in zip(a,b):
        if x==y:
            i+= 1
    percentage = (i/5) * 100
    return percentage

def prepare(filepath):
    IMG_SIZE = 50
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE,IMG_SIZE))
    #plt.imshow(new_array,'gray')
    return new_array.reshape(-1, IMG_SIZE,IMG_SIZE,1)

total_time = 0
score = 0
count = 0
done = []
list_ = preds.predictions('test',CATEGORIES)
CAPTCHA_DIR = list_[0]
for x in CAPTCHA_DIR:
    captchas = random.choice(CAPTCHA_DIR)
    if captchas in done:
        continue
    else:
        done.append(captchas)
        start_time = time.time()
        predict_list= []
        print('\n')
        print('Actual: ',captchas[0][0])

        files = []
        for segments in list_:
            files.append(segments)

        #for segments in files:
        #    filepath = (SEGMENTS_DIR,segments)
        #    prediction = model.predict([prepare(filepath)])
        #    predict_list.append(CATEGORIES[np.argmax(prediction[0])])

        elapsed_time = time.time()-start_time
        time.sleep(captchas[1])
        total_time += elapsed_time
        final_prediction = ''.join(predict_list)
        #if captcha_name[0]== final_prediction:
        #    score +=1
        score = list_[1]
        print('Predicted:',captchas[0][1],'\n')
        print('Elapsed time= ',captchas[1],'s')
        total_time +=captchas[1]
        print('<------------------------------->')    
    

print('Total number of CAPTCHAS broken: ',score)    
print('Total number of CAPTCHAS attempted: ',len(list_[0]))
print('Total time taken= ',total_time,'s')
print('Accuracy= ', (score/len(list_[0])*100))
#model = tf.keras.models.load_model('captcha_alpha.model')
#prediction = model.predict([prepare('3.png')])
#print(len(prediction[0]))
#print(prediction[0])
#print(CATEGORIES[np.argmax(prediction[0])])



Actual:  W3NH
Predicted: 3ZKN 

Elapsed time=  0.11995660783796236 s
<------------------------------->


Actual:  K65AE
Predicted: K65AE 

Elapsed time=  0.07713389839374252 s
<------------------------------->


Actual:  687K9
Predicted: U57M 

Elapsed time=  0.2868424791060552 s
<------------------------------->


Actual:  UJM2
Predicted: UJM2 

Elapsed time=  0.6220111927479844 s
<------------------------------->


Actual:  32QW
Predicted: VDHB 

Elapsed time=  0.6135106794532564 s
<------------------------------->


Actual:  6JHT
Predicted: 6JHT 

Elapsed time=  0.6166515807706134 s
<------------------------------->


Actual:  9PG93
Predicted: K29S 

Elapsed time=  0.7608230043080103 s
<------------------------------->


Actual:  Z3JL
Predicted: Z3JL 

Elapsed time=  0.003355450892842171 s
<------------------------------->


Actual:  687K9
Predicted: 687K9 

Elapsed time=  0.8591054685343618 s
<------------------------------->


Actual:  KCD34
Predicted: KCD34 

Elapsed time=  0.0

Predicted: SZ5F 

Elapsed time=  0.4818294962054521 s
<------------------------------->


Actual:  KG33
Predicted: 23BW 

Elapsed time=  0.45176424734876086 s
<------------------------------->


Actual:  9D7E
Predicted: 9D7E 

Elapsed time=  0.8478642584903671 s
<------------------------------->


Actual:  PP21E
Predicted: MJ6X 

Elapsed time=  0.9262443276467892 s
<------------------------------->


Actual:  W3NH
Predicted: W3NH 

Elapsed time=  0.44954202541009813 s
<------------------------------->


Actual:  UJM2
Predicted: UJM2 

Elapsed time=  0.45669891072077906 s
<------------------------------->


Actual:  WM4UW
Predicted: WM4UW 

Elapsed time=  0.00713506604370262 s
<------------------------------->


Actual:  32QW
Predicted: 32QW 

Elapsed time=  0.5577310216523915 s
<------------------------------->


Actual:  JNAA
Predicted: JNAA 

Elapsed time=  0.6785773628015223 s
<------------------------------->


Actual:  4P14
Predicted: 4P14 

Elapsed time=  0.5846867454837554 s
<--

Predicted: 5GYN 

Elapsed time=  0.7280440173718076 s
<------------------------------->


Actual:  GV18Q
Predicted: GV18Q 

Elapsed time=  0.881095679303337 s
<------------------------------->


Actual:  ZVVC
Predicted: ZVVC 

Elapsed time=  0.14511226277968348 s
<------------------------------->


Actual:  5GYN
Predicted: L5PT 

Elapsed time=  0.11270750936349028 s
<------------------------------->


Actual:  WM4UW
Predicted: WM4UW 

Elapsed time=  0.5254834023332758 s
<------------------------------->


Actual:  6Y6S
Predicted: 6Y6S 

Elapsed time=  0.7827416516882958 s
<------------------------------->


Actual:  PGBZP
Predicted: PGBZP 

Elapsed time=  0.7919174448863194 s
<------------------------------->


Actual:  Q1P7
Predicted: Q1P7 

Elapsed time=  0.8715257737553489 s
<------------------------------->


Actual:  MF7DA
Predicted: MF7DA 

Elapsed time=  0.9769217361385872 s
<------------------------------->


Actual:  XS2E
Predicted: XS2E 

Elapsed time=  0.31912816090191587 s


Predicted: FR46 

Elapsed time=  0.7290794271432284 s
<------------------------------->


Actual:  X1PR3
Predicted: X1PR3 

Elapsed time=  0.5369642510167404 s
<------------------------------->


Actual:  MZZLU
Predicted: GRS2 

Elapsed time=  0.6090486003395827 s
<------------------------------->


Actual:  U8J1
Predicted: U8J1 

Elapsed time=  0.5582819961192697 s
<------------------------------->


Actual:  5YR81
Predicted: 5YR81 

Elapsed time=  0.7946705790864553 s
<------------------------------->


Actual:  5Y5BK
Predicted: USLD 

Elapsed time=  0.8696121136686661 s
<------------------------------->


Actual:  9PG93
Predicted: 9PG93 

Elapsed time=  0.663499457255156 s
<------------------------------->


Actual:  G3H7
Predicted: G3H7 

Elapsed time=  0.5466929857882028 s
<------------------------------->


Actual:  AU4RX
Predicted: CXQV 

Elapsed time=  0.7514231367961658 s
<------------------------------->


Actual:  82SRX
Predicted: 82SRX 

Elapsed time=  0.023942124060484238 